In [0]:
import pandas as pd
import numpy as np
import os

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data_path = "drive/My Drive/kbo_attendance_prediction/data"
data_files = os.listdir(data_path)
keys = ["사직", "창원", "청주", "대구", "대전", "광주", "문학", "포항", "잠실", "수원", "울산"]
rainfall = np.array([os.path.join(data_path,a) for a in os.listdir(data_path) if a.split(" - ")[0] == "rainfall"]).reshape(-1,3)
rainfall = pd.concat([pd.concat([pd.read_csv(f, index_col=0).unstack() for f in r], keys = [2017, 2018, 2019]) for r in rainfall], axis=0, keys = keys)

temperature = np.array([os.path.join(data_path,a) for a in os.listdir(data_path) if a.split(" - ")[0] == "temperature"]).reshape(-1,3)
temperature = pd.concat([pd.concat([pd.read_csv(f, index_col=0).unstack() for f in r], keys = [2017, 2018, 2019]) for r in temperature], axis=0, keys = keys )


In [0]:
rainfall.shape, temperature.shape

((12276,), (12276,))

In [0]:
rainfall = rainfall.fillna(0.0)
rainfall = rainfall.reset_index()
rainfall.columns = ["구장", "연", "월", "일","강수량"]
rainfall = rainfall.replace("월|일","", regex=True)
rainfall.head()

,구장,연,월,일,강수량
0,사직,2017,1,1,0.0
1,사직,2017,1,2,0.0
2,사직,2017,1,3,0.0
3,사직,2017,1,4,0.0
4,사직,2017,1,5,0.0


In [0]:
temperature = temperature.fillna(0.0)
temperature = temperature.reset_index()
temperature.columns = ["구장", "연", "월", "일","기온"]
temperature = temperature.replace("월|일","", regex=True)
temperature.head()

,구장,연,월,일,기온
0,사직,2017,1,1,2.7
1,사직,2017,1,2,4.1
2,사직,2017,1,3,2.6
3,사직,2017,1,4,2.8
4,사직,2017,1,5,4.1


In [0]:
weather = pd.merge(rainfall, temperature)
weather.head()

,구장,연,월,일,강수량,기온
0,사직,2017,1,1,0.0,2.7
1,사직,2017,1,2,0.0,4.1
2,사직,2017,1,3,0.0,2.6
3,사직,2017,1,4,0.0,2.8
4,사직,2017,1,5,0.0,4.1


In [0]:
weather.shape

(12276, 6)

In [0]:
attendance_2017 = pd.read_csv(os.path.join(data_path, "kbo_attendance_status_2017.csv"), thousands = ',')
attendance_2018 = pd.read_csv(os.path.join(data_path, "kbo_attendance_status_2018.csv"), thousands = ',')
attendance_2019 = pd.read_csv(os.path.join(data_path, "kbo_attendance_status_2019.csv"), thousands = ',')
test = pd.read_csv(os.path.join(data_path, "kbo_test.csv"))

attendance_2017 = attendance_2017.replace("넥센","키움")
attendance_2018 = attendance_2018.replace("넥센","키움")
(attendance_2017.shape, attendance_2018.shape, attendance_2019.shape, test.shape)

((720, 6), (720, 6), (158, 6), (15, 5))

In [0]:
attendance_avg = pd.read_csv(os.path.join(data_path,"kbo_attendance_avg.csv"), index_col=0)
attendance_avg.columns = ["삼성", "KIA", "롯데", "LG", "두산", "한화", "SK", "키움", "NC", "KT"]
attendance_avg.head()

,삼성,KIA,롯데,LG,두산,한화,SK,키움,NC,KT
연도,,,,,,,,,,
2018,10449,11968,12523,15398,15445,10196,14406,6314,6151,9286
2017,9790,14234,14424,15762,15206,8240,12396,9714,7377,9535


In [0]:
for h in attendance_2017["홈"].unique():
    attendance_2017.loc[attendance_2017["홈"]== h , "팀별 관중수"] = attendance_avg[h][2017]
for a in attendance_2017["방문"].unique():
    attendance_2017.loc[attendance_2017["방문"]== a , "팀별 관중수"] += attendance_avg[a][2017]
attendance_2017["팀별 관중수"] /= 2
attendance_2017.head()

,날짜,요일,홈,방문,구장,관중수,팀별 관중수
0,2017/03/31,금,두산,한화,잠실,21121,11723.0
1,2017/03/31,금,삼성,KIA,대구,13505,12012.0
2,2017/03/31,금,SK,KT,문학,13649,10965.5
3,2017/03/31,금,키움,LG,고척,8013,12738.0
4,2017/03/31,금,NC,롯데,마산,11000,10900.5


In [0]:
for h in attendance_2018["홈"].unique():
    attendance_2018.loc[attendance_2018["홈"]== h , "팀별 관중수"] = attendance_avg[h][2018]
for a in attendance_2018["방문"].unique():
    attendance_2018.loc[attendance_2018["방문"]== a , "팀별 관중수"] += attendance_avg[a][2018]
attendance_2018["팀별 관중수"] /= 2
attendance_2018.head()

,날짜,요일,홈,방문,구장,관중수,팀별 관중수
0,2018/03/24,토,키움,한화,고척,15055,8255.0
1,2018/03/24,토,KIA,KT,광주,20500,10627.0
2,2018/03/24,토,NC,LG,마산,11000,10774.5
3,2018/03/24,토,SK,롯데,문학,25000,13464.5
4,2018/03/24,토,두산,삼성,잠실,25000,12947.0


In [0]:
for h in attendance_2019["홈"].unique():
    attendance_2019.loc[attendance_2019["홈"]== h , "팀별 관중수"] = (attendance_avg[h][2018] + attendance_avg[h][2017])/2
for a in attendance_2019["방문"].unique():
    attendance_2019.loc[attendance_2019["방문"]== a , "팀별 관중수"] += (attendance_avg[a][2018] + attendance_avg[a][2017])/2
attendance_2019["팀별 관중수"] /= 2
attendance_2019.head()

,날짜,요일,홈,방문,구장,관중수,팀별 관중수
0,2019/03/23,토,두산,한화,잠실,25000,12271.75
1,2019/03/23,토,SK,KT,문학,21909,11405.75
2,2019/03/23,토,KIA,LG,광주,20500,14340.50
3,2019/03/23,토,NC,삼성,창원,22112,8441.75
4,2019/03/23,토,롯데,키움,사직,24500,10743.75


In [0]:
for h in test["홈"].unique():
    test.loc[test["홈"]== h , "팀별 관중수"] = (attendance_avg[h][2018] + attendance_avg[h][2017])/2
for a in test["방문"].unique():
    test.loc[test["방문"]== a , "팀별 관중수"] += (attendance_avg[a][2018] + attendance_avg[a][2017])/2
test["팀별 관중수"] /= 2
test.head()

,날짜,요일,홈,방문,구장,팀별 관중수
0,2019-05-07,화,두산,KIA,잠실,14213.25
1,2019-05-07,화,SK,한화,문학,11309.50
2,2019-05-07,화,삼성,NC,대구,8441.75
3,2019-05-07,화,KT,롯데,수원,11442.00
4,2019-05-07,화,키움,LG,고척,11797.00


In [0]:
train = pd.concat([attendance_2017, attendance_2018, attendance_2019])

for i in train["구장"].unique():
    train.loc[train["구장"]==i, "구장 평균 관중수"] = train[train["구장"]==i]["관중수"].mean()
train.head()

,날짜,요일,홈,방문,구장,관중수,팀별 관중수,구장 평균 관중수
0,2017/03/31,금,두산,한화,잠실,21121,11723.0,15296.246875
1,2017/03/31,금,삼성,KIA,대구,13505,12012.0,10399.255172
2,2017/03/31,금,SK,KT,문학,13649,10965.5,13330.141104
3,2017/03/31,금,키움,LG,고척,8013,12738.0,7815.471698
4,2017/03/31,금,NC,롯데,마산,11000,10900.5,6763.840278


In [0]:
for i in test["구장"].unique():
    test.loc[test["구장"]==i, "구장 평균 관중수"] = train[train["구장"]==i]["관중수"].mean()
test.head()

,날짜,요일,홈,방문,구장,팀별 관중수,구장 평균 관중수
0,2019-05-07,화,두산,KIA,잠실,14213.25,15296.246875
1,2019-05-07,화,SK,한화,문학,11309.50,13330.141104
2,2019-05-07,화,삼성,NC,대구,8441.75,10399.255172
3,2019-05-07,화,KT,롯데,수원,11442.00,9316.474684
4,2019-05-07,화,키움,LG,고척,11797.00,7815.471698


In [0]:
total = pd.concat([train, test])
total.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,관중수,구장,구장 평균 관중수,날짜,방문,요일,팀별 관중수,홈
0,21121.0,잠실,15296.246875,2017/03/31,한화,금,11723.0,두산
1,13505.0,대구,10399.255172,2017/03/31,KIA,금,12012.0,삼성
2,13649.0,문학,13330.141104,2017/03/31,KT,금,10965.5,SK
3,8013.0,고척,7815.471698,2017/03/31,LG,금,12738.0,키움
4,11000.0,마산,6763.840278,2017/03/31,롯데,금,10900.5,NC


In [0]:
weather_g = weather[weather["구장"]=="잠실"].copy().replace("잠실","고척")
weather_m = weather[weather["구장"]=="창원"].copy().replace("창원","마산")

weather = pd.concat([weather, weather_g, weather_m])
weather.shape, weather_g.shape, weather_m.shape

((14508, 6), (1116, 6), (1116, 6))

In [0]:
datetime = pd.to_datetime(total["날짜"]).dt
total.drop(["요일", "날짜"], axis=1, inplace=True)
total.insert(0, "연", datetime.year)
total.insert(1, "월", datetime.month)
total.insert(2, "일", datetime.day)
total.insert(3, "요일",datetime.dayofweek)
total.tail()

,연,월,일,요일,관중수,구장,구장 평균 관중수,방문,팀별 관중수,홈
10,2019,5,9,3,NaN,잠실,15296.246875,KIA,14213.25,두산
11,2019,5,9,3,NaN,문학,13330.141104,한화,11309.50,SK
12,2019,5,9,3,NaN,대구,10399.255172,NC,8441.75,삼성
13,2019,5,9,3,NaN,수원,9316.474684,롯데,11442.00,KT
14,2019,5,9,3,NaN,고척,7815.471698,LG,11797.00,키움


In [0]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1613 entries, 0 to 14
Data columns (total 10 columns):
연            1613 non-null int64
월            1613 non-null int64
일            1613 non-null int64
요일           1613 non-null int64
관중수          1598 non-null float64
구장           1613 non-null object
구장 평균 관중수    1613 non-null float64
방문           1613 non-null object
팀별 관중수       1613 non-null float64
홈            1613 non-null object
dtypes: float64(3), int64(4), object(3)
memory usage: 138.6+ KB


In [0]:
weather[["연", "월", "일"]] = rainfall[["연", "월", "일"]].astype(int)
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14508 entries, 0 to 2231
Data columns (total 6 columns):
구장     14508 non-null object
연      14508 non-null int64
월      14508 non-null int64
일      14508 non-null int64
강수량    14508 non-null float64
기온     14508 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 793.4+ KB


In [0]:
total = pd.merge(total, weather)
total.tail()

,연,월,일,요일,관중수,구장,구장 평균 관중수,방문,팀별 관중수,홈,강수량,기온
1608,2019,5,9,3,NaN,잠실,15296.246875,KIA,14213.25,두산,0.0,11.7
1609,2019,5,9,3,NaN,문학,13330.141104,한화,11309.50,SK,0.0,16.1
1610,2019,5,9,3,NaN,대구,10399.255172,NC,8441.75,삼성,0.0,12.9
1611,2019,5,9,3,NaN,수원,9316.474684,롯데,11442.00,KT,0.0,0.0
1612,2019,5,9,3,NaN,고척,7815.471698,LG,11797.00,키움,0.0,11.7


In [0]:
teams = pd.get_dummies(total["홈"]) + pd.get_dummies(total["방문"])
location = pd.get_dummies(total["구장"])

del total["홈"]
del total["방문"]
del total["구장"]
total = pd.concat([total, teams, location], axis = 1)
total.tail()

,연,월,일,요일,관중수,구장 평균 관중수,팀별 관중수,강수량,기온,KIA,KT,LG,NC,SK,두산,롯데,삼성,키움,한화,고척,광주,대구,대전,마산,문학,사직,수원,울산,잠실,창원,청주,포항
1608,2019,5,9,3,NaN,15296.246875,14213.25,0.0,11.7,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1609,2019,5,9,3,NaN,13330.141104,11309.50,0.0,16.1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1610,2019,5,9,3,NaN,10399.255172,8441.75,0.0,12.9,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1611,2019,5,9,3,NaN,9316.474684,11442.00,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1612,2019,5,9,3,NaN,7815.471698,11797.00,0.0,11.7,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
total["isRain"] = (total["강수량"]>0.0).astype(int)
total["isWeekend"] = (total["요일"]>3).astype(int)
total.tail()

,연,월,일,요일,관중수,구장 평균 관중수,팀별 관중수,강수량,기온,KIA,KT,LG,NC,SK,두산,롯데,삼성,키움,한화,고척,광주,대구,대전,마산,문학,사직,수원,울산,잠실,창원,청주,포항,isRain,isWeekend
1608,2019,5,9,3,NaN,15296.246875,14213.25,0.0,11.7,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1609,2019,5,9,3,NaN,13330.141104,11309.50,0.0,16.1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1610,2019,5,9,3,NaN,10399.255172,8441.75,0.0,12.9,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1611,2019,5,9,3,NaN,9316.474684,11442.00,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1612,2019,5,9,3,NaN,7815.471698,11797.00,0.0,11.7,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
total.loc[total["강수량"]>10, "강수량"] = 10
total.tail()

,연,월,일,요일,관중수,구장 평균 관중수,팀별 관중수,강수량,기온,KIA,KT,LG,NC,SK,두산,롯데,삼성,키움,한화,고척,광주,대구,대전,마산,문학,사직,수원,울산,잠실,창원,청주,포항,isRain,isWeekend
1608,2019,5,9,3,NaN,15296.246875,14213.25,0.0,11.7,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1609,2019,5,9,3,NaN,13330.141104,11309.50,0.0,16.1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1610,2019,5,9,3,NaN,10399.255172,8441.75,0.0,12.9,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1611,2019,5,9,3,NaN,9316.474684,11442.00,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1612,2019,5,9,3,NaN,7815.471698,11797.00,0.0,11.7,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
train.shape, test.shape, total.shape

((1598, 8), (15, 7), (1613, 34))

In [0]:
train_data = total[:len(train)]

total.loc[len(train):, "기온"] = 15.0
test_data = total[len(train):]

train_data.shape, test_data.shape, total.shape

((1598, 34), (15, 34), (1613, 34))

In [0]:
X_tr = train_data.drop(["관중수"], axis=1).values
Y_tr = train_data["관중수"].values

X_test = test_data.drop(["관중수"], axis=1).values

X_tr.shape, Y_tr.shape, X_test.shape


((1598, 33), (1598,), (15, 33))

In [0]:
from sklearn.ensemble import RandomForestRegressor 

In [0]:
model_rf = RandomForestRegressor(n_estimators=1000,  criterion='mse', max_features="auto", bootstrap=True, 
                               oob_score=True, random_state=42 )
model_rf.fit(X_tr, Y_tr)
prediction = model_rf.predict(X_test)
model_rf.score(X_tr, Y_tr), prediction

(0.9594404099928407,
 array([11934.116, 12716.477,  6003.139,  7186.472,  4678.14 , 12185.291,
        12221.906,  6320.605,  7395.587,  4761.597, 12730.116, 12296.589,
         6498.204,  7614.722,  4846.037]))

In [0]:
test.loc[:,"관중수"] = prediction.astype(int)
test

,날짜,요일,홈,방문,구장,팀별 관중수,구장 평균 관중수,관중수
0,2019-05-07,화,두산,KIA,잠실,14213.25,15296.246875,11934
1,2019-05-07,화,SK,한화,문학,11309.50,13330.141104,12716
2,2019-05-07,화,삼성,NC,대구,8441.75,10399.255172,6003
3,2019-05-07,화,KT,롯데,수원,11442.00,9316.474684,7186
4,2019-05-07,화,키움,LG,고척,11797.00,7815.471698,4678
5,2019-05-08,수,두산,KIA,잠실,14213.25,15296.246875,12185
6,2019-05-08,수,SK,한화,문학,11309.50,13330.141104,12221
7,2019-05-08,수,삼성,NC,대구,8441.75,10399.255172,6320
8,2019-05-08,수,KT,롯데,수원,11442.00,9316.474684,7395
9,2019-05-08,수,키움,LG,고척,11797.00,7815.471698,4761


In [0]:
stadium = test["구장"].unique()
attendance = [ test[test["구장"]==x]["관중수"].sum() for x in stadium]
result = pd.DataFrame(attendance, columns=["총 관중수"], index = stadium)
result

,총 관중수
잠실,36849
문학,37233
대구,18821
수원,22195
고척,14285


In [0]:
with pd.ExcelWriter('result2.xlsx') as writer:
    result.to_excel(writer, sheet_name='구장별 총 관중수')
    test.to_excel(writer, sheet_name='경기별 관중수')